# 사전 준비

In [ ]:
!pip install transformers
!pip install datasets

**네이버 뉴스 요약 데이터 불러오기**

In [4]:
from datasets import load_dataset

datasets = load_dataset("daekeun-ml/naver-news-summarization-ko")

  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
})

**KoBART 모델과 Tokenizer 불러오기**

In [6]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


# 데이터 구축

**전처리 함수 정의**


In [ ]:
!pip install soynlp
!pip install emoji==1.7.0

In [8]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
special_symbol = re.compile(
    r'([.,?!/@$%~％·∼()\x21-\x2F\x3A-\x40\x5B-\x60\x7B-\x7E])\1{1,}')

In [9]:
def clean(x):
    x = pattern.sub(' ', x)                     # 일반적으로 사용하는 특수문자, 영어, 한글, emoji제외 공백으로 치환
    x = url_pattern.sub('', x)                  # URL 제거
    x = special_symbol.sub('\1'*1, x)          # 반복되는 특수문자의 축약 횟수 1개로 줄임
    x = x.strip()                               # 문자의 시작과 끝에서 공백제거
    x = repeat_normalize(x, num_repeats=2)      # 반목되는 문자의 축약 횟수 2개로 줄임
    
    return x

**데이터셋 구축**

In [10]:
import numpy as np
from torch.utils.data import Dataset

class SummarySet(Dataset):
    def __init__(self, docs, tokenizer, max_len, ignore_id = -100):
        super().__init__()
        self.docs = docs
        self.tokenizer = tokenizer
        self.max_len = max_len

        self.bos_id = tokenizer.bos_token_id
        self.eos_id = tokenizer.eos_token_id
        self.pad_id = tokenizer.pad_token_id
        self.ignore_id = ignore_id      # BartForConditionalGeneration의 labels 입력 index에 포함

    # padding 및 데이터 size 일치화 함수 (input_ids)
    def add_padding(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.pad_id]*(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]
            
        return inputs

    # padding 및 데이터 size 일치화 함수 (labels)
    def add_ignored(self, inputs):
        if len(inputs) < self.max_len:
            ignored = np.array([self.ignore_id]*(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, ignored])
        else:
            inputs = inputs[:self.max_len]

        return inputs

    def __len__(self):  
        return self.docs.num_rows

    def __getitem__(self, idx):
        instance = self.docs[idx]

        input_ids = self.tokenizer.encode(clean(instance['document']))
        input_ids = self.add_padding(input_ids)

        labels = self.tokenizer.encode(clean(instance['summary']))
        labels.append(self.eos_id)

        dec_input_ids = [self.eos_id]       # BART 디코더 입력의 시작은 eos토큰의 인덱스로 시작
        dec_input_ids += labels[:-1]
        dec_input_ids = self.add_padding(dec_input_ids)

        labels = self.add_ignored(labels)

        return {'input_ids': np.array(input_ids, dtype=np.int_),
                'decoder_input_ids': np.array(dec_input_ids, dtype=np.int_),
                'labels': np.array(labels, dtype=np.int_)}

In [11]:
train_set = SummarySet(datasets["train"], tokenizer, max_len=512)
val_set = SummarySet(datasets["validation"], tokenizer, max_len=512)
test_set = SummarySet(datasets["test"], tokenizer, max_len=512)

**데이터로더 구축**

In [12]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_set, shuffle=False, batch_size=8)      # 22194 = 2*(3^4)*137  64size일때 마지막 배치의 데이터 50개, 32size일떄 18개
val_dataloader = DataLoader(val_set, shuffle=False, batch_size=8)      # 2740 = 2^2*5*137  32size일때 마지막 배치의 데이터 20개, 64size일때 52개
test_dataloader = DataLoader(test_set, shuffle=False, batch_size=8)      # 2466 = 2*3^2*137  16, 32size 모두 마지막 배치의 데이터 2개

In [13]:
next(iter(train_dataloader))

{'input_ids': tensor([[26001, 15530, 14516,  ...,     3,     3,     3],
         [14111, 11763,  1700,  ...,     3,     3,     3],
         [14036, 13125, 17350,  ..., 14025, 14053, 14318],
         ...,
         [14136, 15243, 15262,  ...,     3,     3,     3],
         [14454, 14809, 15809,  ...,     3,     3,     3],
         [14287, 16598, 12325,  ...,     3,     3,     3]]),
 'decoder_input_ids': tensor([[    1, 14516, 18115,  ...,     3,     3,     3],
         [    1, 14705, 18633,  ...,     3,     3,     3],
         [    1, 14025, 14025,  ...,     3,     3,     3],
         ...,
         [    1, 16476, 16476,  ...,     3,     3,     3],
         [    1, 14454, 20586,  ...,     3,     3,     3],
         [    1, 16969, 16170,  ...,     3,     3,     3]]),
 'labels': tensor([[14516, 18115, 15736,  ...,  -100,  -100,  -100],
         [14705, 18633, 22751,  ...,  -100,  -100,  -100],
         [14025, 14025, 13699,  ...,  -100,  -100,  -100],
         ...,
         [16476, 16476, 1

In [14]:
next(iter(val_dataloader))

{'input_ids': tensor([[14245, 18597, 14465,  ...,     3,     3,     3],
         [19813, 26114, 14116,  ...,     3,     3,     3],
         [21985, 23170, 19235,  ...,     3,     3,     3],
         ...,
         [18115, 17214, 15062,  ..., 15358,  9908, 12332],
         [15981, 14347, 17757,  ...,     3,     3,     3],
         [14322, 14053, 14106,  ...,     3,     3,     3]]),
 'decoder_input_ids': tensor([[    1, 14195, 16601,  ...,     3,     3,     3],
         [    1, 14025, 14898,  ...,     3,     3,     3],
         [    1, 21985, 23170,  ...,     3,     3,     3],
         ...,
         [    1, 14235, 14150,  ...,     3,     3,     3],
         [    1, 14053, 14036,  ...,     3,     3,     3],
         [    1, 15976,   373,  ...,     3,     3,     3]]),
 'labels': tensor([[14195, 16601, 14689,  ...,  -100,  -100,  -100],
         [14025, 14898, 16732,  ...,  -100,  -100,  -100],
         [21985, 23170, 19235,  ...,  -100,  -100,  -100],
         ...,
         [14235, 14150, 2

In [15]:
next(iter(test_dataloader))

{'input_ids': tensor([[14360, 11790, 19958,  ...,     3,     3,     3],
         [14188, 13679, 18904,  ..., 15453, 17613, 19754],
         [21473, 13173,  9932,  ..., 23413, 14030, 14441],
         ...,
         [14572, 20165,   283,  ...,     3,     3,     3],
         [18168, 21207, 20279,  ...,     3,     3,     3],
         [14469, 28863, 16391,  ..., 24448, 16779,  9545]]),
 'decoder_input_ids': tensor([[    1, 14360, 21622,  ...,     3,     3,     3],
         [    1, 14188, 13679,  ...,     3,     3,     3],
         [    1, 14029, 15582,  ...,     3,     3,     3],
         ...,
         [    1, 16415, 14702,  ...,     3,     3,     3],
         [    1, 18168, 14030,  ...,     3,     3,     3],
         [    1, 14469, 28863,  ...,     3,     3,     3]]),
 'labels': tensor([[14360, 21622, 14360,  ...,  -100,  -100,  -100],
         [14188, 13679, 18904,  ...,  -100,  -100,  -100],
         [14029, 15582, 12258,  ...,  -100,  -100,  -100],
         ...,
         [16415, 14702, 1

# 모델 학습

**모델 파라미터 설정**

In [ ]:
import torch
from transformers import get_linear_schedule_with_warmup

# GPU 가속을 사용할 수 있으면 device를 cuda로 설정하고, 아니면 cpu로 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 3
learning_rate = 1e-4

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# 학습 스케줄러 설정
scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,
                                            num_training_steps=len(train_dataloader),
                                            num_warmup_steps=0)

step = 0
eval_steps = len(train_dataloader)

In [17]:
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

**KoBART 학습 진행**

In [18]:
torch.cuda.empty_cache()

In [19]:
from tqdm.auto import tqdm as tqdm_auto

for epoch in range(num_epochs):
    loss = 0
    train_loss = 0.0
    
    model.train()
    for batch in tqdm_auto(train_dataloader, mininterval=0.01, leave=True):
        optimizer.zero_grad()     # 그래디언트 초기화

        # 배치를 GPU로 복사
        inputs = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**inputs)   # 모형으로 결과 예측

        loss = outputs.loss
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        # scheduler.step()

        step += 1
        if step % eval_steps == 0:  # eval_steps 마다 loss를 출력

            with torch.no_grad():   # 학습 X (그래디언트 계산 X)
                val_loss = 0
                model.eval()        # 평가모드로 전환

                for val_batch in tqdm_auto(val_dataloader, mininterval=0.01, leave=True):

                    # 배치를 GPU로 복사
                    inputs = {k: v.to(device) for k, v in batch.items()}
                    val_outputs = model(**inputs)     # 모형으로 결과 예측

                    loss = outputs.loss 
                    val_loss += loss.item()

                avg_val_loss = val_loss / len(val_dataloader)

            print('Step %d, validation loss: %.4f' % (step, avg_val_loss))
            
        avg_train_loss = train_loss / len(train_dataloader)

    print('epoch %d, train loss: %.4f \n' % (epoch, avg_train_loss))
    torch.cuda.empty_cache()

  0%|          | 0/2775 [00:00<?, ?it/s]

  0%|          | 0/309 [00:00<?, ?it/s]

Step 2775, validation loss: 0.6964
epoch 0, train loss: 0.6498 



  0%|          | 0/2775 [00:00<?, ?it/s]

  0%|          | 0/309 [00:00<?, ?it/s]

Step 5550, validation loss: 0.6598
epoch 1, train loss: 0.4715 



  0%|          | 0/2775 [00:00<?, ?it/s]

  0%|          | 0/309 [00:00<?, ?it/s]

Step 8325, validation loss: 0.3570
epoch 2, train loss: 0.3657 



In [20]:
model.save_pretrained('/content/drive/Othercomputers/내 컴퓨터/Article_summary/article_summary_model_no_schedular.pt')

**모델 테스트**

In [6]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

def load_model():
    saved_model = BartForConditionalGeneration.from_pretrained('/content/drive/Othercomputers/내 컴퓨터/Article_summary/article_summary_model_no_schedular.pt')

    return saved_model

In [7]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [10]:
import torch

fine_tuned_model = load_model()

while 1:
    document = input("기사 입력 > \n").strip()
    # quit 입력시 종료
    if document == "quit":
        break

    input_ids = tokenizer.encode(document)
    input_ids = torch.tensor(input_ids)
    input_ids = input_ids.unsqueeze(0)

    output = fine_tuned_model.generate(input_ids, 
                                       eos_token_id=1, 
                                       max_length=512
                                       )
    output = tokenizer.decode(output[0], skip_special_tokens=True)

    print(f'기사 요약 결과 > \n{output}')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


기사 입력 > 
인공지능(AI) 기술에 대한 국가 지원과 국민 기본권 보호 등을 명문화한 법안이 국회 상임위원회 소위원회 문턱을 넘었다.  국회 과학기술정보방송통신위원회 소속 정필모 더불어민주당 의원은 대표 발의한 '인공지능산업 육성 및 신뢰 기반 조성 등에 관한 법률안(AI법 제정안)'이 14일 정보통신방송법안심사소위원회를 통과했다고 이날 밝혔다.  이 제정안에 따르면 과학기술정보통신부는 투자 및 인력양성 등 AI 정책 기본 방향을 담은 'AI 기본계획'을 3년마다 수립해야 한다. 정책과 예산 등을 심의하기 위한 AI위원회도 구성해야 한다. 위원회는 민간 위원을 과반수로 하고 국무총리와 대통령이 위촉하는 민간위원을 공동 위원장으로 임명하도록 했다.  AI위원회 내에는 각계 의견을 수렴하고 공정성과 투명성 등을 위한 논의와 연구를 위한 민간 전문가로 구성된 '신뢰성 전문위원회'를 두도록 했다.  해당 제정안은 AI 기술에 대해 '우선 허용·사후 규제'를 원칙을 세웠다. 누구든지 AI기술과 알고리즘을 연구·개발하고 서비스로 출시할 수 있도록 보장했다. 다만 생명과 안전, 기본권에 중대한 영향을 미칠 수 있는 경우 사업자는 이용자에게 이를 알리고 신뢰성 확보 조치 등을 취해야 한다.  또한 AI기술과 산업에 대한 안전성과 신뢰성 제고도 기본 원칙으로 삼았다. AI로 인한 사회·경제·문화와 일상생활 등 변화에 국민이 안정적으로 적응할 수 있도록 국가와 지자체가 정책을 세우도록 했다.  정필모 의원은 "최근 챗GPT가 최첨단 기술 수준을 선보이며 전 세계에 충격을 던졌다"며 "국내 AI 기술 발전 기반을 마련하고 국가 역량을 집중하기 위해 AI법 제정이 시급하다"고 강조했다.
기사 요약 결과 > 
정필모 더불어민주당 의원은 14일 정보통신방송법안심사소위원회를 통과한 '인공지능산업 육성 및 신뢰 기반 조성 등에 관한 법률안' 이 통과했다고 이날 밝혔으며 이 제정안에 따르면 과학기술정보통신부는 투자 및 인력양성 등 AI 정책 기본 방향을 담은 'AI 기본계획'을 3년마다